# **LASSO REGRESSION**

## **A. CONCEPT**

***Lasso Regression*** is a type of linear regression that adds an L1 penalty term to the objective function, encouraging sparsity in the model. The main idea behind Lasso is not to just fit the best possible line (like in standard linear regression) but to also **shrink some of the coefficients towards zero**, eliminating less important features (variables). This makes Lasso particularly useful for feature selection and prevent overfitting.

### **1. Objective of Lasso Regression**

The goal of Lasso Regression is to minimize the following loss function:

$Minimize \sum_{i=1}^{n} (y_i - \hat{y_i})^2 + \lambda \sum_{j=1}^{p} \lvert{\beta_j}\rvert$

where:

- $y_i$: the actual value of the i-th observation.
- $\hat{y_i}$: the predicted value for the i-th observation (based on the model).
- $\beta_j$: the model coefficients (parameters/weights).
- $\lambda$: the regularization parameter (controls the strength of the penalty).
- $p$: the numbers of features.

The first part of the equation $\sum_{i=1}^{n} (y_i - \hat{y_i})^2$ is Residual Sum of Squares RSS, which measures how well the model fits the data.

The second part $\lambda \sum_{j=1}^{p} \lvert{\beta_j}\rvert$ is the **L1 regularization term**, where:

- The sum $\sum_{j=1}^{p} \lvert{\beta_j}\rvert$ is the **L1 norm** of the coefficients, which is the sum of the absolute values of the coefficients.
- The regularization $\lambda$ controls **the degree of shrinkage**: higher values of $\lambda$ impose stronger regularization (penalizing larger coefficients more), which leads to sparser models (more coefficients set to zero).

### **2. Gradient Descent for Lasso Regression**

To train Lasso Regression, we need to use Gradient Descent, which update the model parameters iteratively to minimize the cost function.

$\beta_j \leftarrow \beta_j$ - \alpha \frac{\partial J(\beta)}{\partial \beta_j}$

The Lasso Cost Function is a combination of the quadratic loss function (squared error) and L1 penalty term. To find the gradient (derivative) with respect to the coefficient $\beta_j$, both components will be calculated seperately.

#### *2.1. Derivative of RSS*

The RSS cost function is:

$RSS = \sum_{i=1}^{N} (y_i - \hat{y_i})^2$

The derivative of RSS with respect to $\beta_j$ is:

$\frac{\partial RSS}{\partial \beta_j} = -2 \sum_{i=1}^{N} x_{ij}$

where:
- $x_{ij}$: the value of the j-th feature for the i-th observation.
- $\hat{y_i}$: the predicted value for the i-th observation.

#### *2.2. Derivative of the L1 Regularization Term:*

The L1 regularization term is:

$L1 = \lambda \sum_{j=1}^{p} \lvert \beta_j \rvert$

The derivative of $\lvert \beta_j \rvert$ with respect to $\beta_j$ is:

$\frac{\partial \lvert \beta_j \rvert}{\partial \beta_j} = sgn(\beta_j)$

where $sgn(\beta_j)$ is the **sign function**:

$sgn(\beta_j) = \begin{Bmatrix} 1 & if & \beta_j > 0\\ -1 & if & \beta_j < 0\\ 0 & if & \beta_j = 0\end{Bmatrix}$

So, the derivative of the L1 regularization term with respect to $\beta_j$ is:

$\frac{\partial L1}{\partial \beta_j} = \lambda sgn(\beta_j)$

#### *2.3. Gradient of Lasso Cost Function*

Combining the Derivative of RSS and the Derivative of the L1 Regularization Term, we get the Gradient of Lasso Cost Function:

$\frac{\partial J(\beta)}{\partial \beta_j} = -2 \sum_{i=1}^{N} (y_i - \hat{y_i})x_{ij} + \lambda sgn(\beta_j)$

where:
- The first term comes from RSS (squared error).
- The second term comes from the L1 Regularization.

### **3. Lasso vs Ridge**

- **Ridge**: Uses an **L2 penalty** (sum of squared coefficients). Ridge tends to shrink coefficients but does not set them to zero, making all features stay remained in the model.

- **Lasso**: Uses an **L1 penalty** (sum of absolute coefficients), which can shrink coefficients to *exactly zero*, making Lasso a good choice for feature selection.

## **B. IMPLEMENTATIONS**

### **0. Preparing data**

The dataset for implementing LASSO is going to be the **Auto MPG Dataset**

This dataset contains data on the fuel consumption (miles per gallon) of various car models, along with other attributes like engine displacement, horsepower, weight, acceleration, and model year.

- **Dataset Source**: Auto MPG Dataset

- **Labels**: Continuous values representing miles per gallon (mpg).

- **Scope**: Covers different car models with attributes such as engine displacement, horsepower, and weight.

- **Size**: 398 samples, each with 8 attributes.

- **Language**: N/A (numerical data).

In [110]:
# Importing dependecies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from time import time

In [111]:
# Loading the dataset
DATA_PATH = 'data/auto-mpg[1].csv'
data = pd.read_csv(DATA_PATH)
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [112]:
# Data Preprocessing
# Dropping car name column
data = data.drop('car name', axis=1) if 'car name' in data.columns else data

# Dropping rows with missing values
data = data.dropna()

# Dropping rows with "?" values
data = data[data.horsepower != '?']

# Splitting the data into features and target
X = data.drop('mpg', axis=1)
y = data['mpg']

print(X.head(), "\n", X.shape)
print(y.head(), "\n", y.shape)

   cylinders  displacement horsepower  weight  acceleration  model year  \
0          8         307.0        130    3504          12.0          70   
1          8         350.0        165    3693          11.5          70   
2          8         318.0        150    3436          11.0          70   
3          8         304.0        150    3433          12.0          70   
4          8         302.0        140    3449          10.5          70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1   
 (392, 7)
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64 
 (392,)


In [113]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train Shape: {X_train.shape}") 
print(f"X_test Shape: {X_test.shape}")
print(f"y_train Shape: {y_train.shape}") 
print(f"y_test Shape: {y_test.shape}")

X_train Shape: (313, 7)
X_test Shape: (79, 7)
y_train Shape: (313,)
y_test Shape: (79,)


In [114]:
# Normalization
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X)
X_test = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y.values.reshape(-1, 1)).squeeze()
y_test = scaler_y.transform(y_test.values.reshape(-1, 1)).squeeze()

print(X[:5], "\n", X.shape)
print(y[:5], "\n", y.shape)

   cylinders  displacement horsepower  weight  acceleration  model year  \
0          8         307.0        130    3504          12.0          70   
1          8         350.0        165    3693          11.5          70   
2          8         318.0        150    3436          11.0          70   
3          8         304.0        150    3433          12.0          70   
4          8         302.0        140    3449          10.5          70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1   
 (392, 7)
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64 
 (392,)


In [115]:
# Checking the mean and standard deviation of the features
print("Mean of features: ", X_train.mean(axis=0))
print("Standard deviation of features: ", X_train.std(axis=0))

Mean of features:  [-1.08756541e-16 -7.25043608e-17 -1.81260902e-16 -1.81260902e-17
  4.35026165e-16 -1.16006977e-15  1.35945676e-16]
Standard deviation of features:  [1. 1. 1. 1. 1. 1. 1.]


In [116]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=1e-5)  # Set a small threshold for near-zero variance
X_train = selector.fit_transform(X_train)
X_test = selector.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(392, 7)
(79, 7)


### **1. Training model**

In [117]:
# Setting up hyperparameters
EPOCHS = 1000
LAMBDA = 0.1
LEARNING_RATE = 0.0001

In [118]:
# Adding intercept term to X
X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train)) if X_train.shape[1] == 7 else X_train
X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test)) if X_test.shape[1] == 7 else X_test

print(X_train[:5], "\n", X_train.shape)
print(X_test[:5], "\n", X_test.shape)

[[ 1.          1.48394702  1.07728956  0.66413273  0.62054034 -1.285258
  -1.62531533 -0.71664105]
 [ 1.          1.48394702  1.48873169  1.57459447  0.84333403 -1.46672362
  -1.62531533 -0.71664105]
 [ 1.          1.48394702  1.1825422   1.18439658  0.54038176 -1.64818924
  -1.62531533 -0.71664105]
 [ 1.          1.48394702  1.04858429  1.18439658  0.53684535 -1.285258
  -1.62531533 -0.71664105]
 [ 1.          1.48394702  1.02944745  0.92426466  0.5557062  -1.82965485
  -1.62531533 -0.71664105]] 
 (392, 8)
[[ 1.         -0.86401356 -0.94164742 -0.92267201 -0.92958509  0.89232939
  -1.08169451  0.52638236]
 [ 1.         -0.86401356 -0.70243687  0.27393484 -0.21523071  0.05758756
   0.54916798  0.52638236]
 [ 1.         -0.86401356 -0.98948953 -1.15679075 -1.38813931  0.31163942
   0.54916798  1.76940577]
 [ 1.         -0.86401356 -0.98948953 -0.89665882 -1.20542491  1.79965748
  -1.35350492 -0.71664105]
 [ 1.         -0.86401356 -0.52063686 -0.48044774 -0.22112473  0.02129443
   1.6364

In [119]:
# Initializing weights
np.random.seed(42)
beta = np.zeros(X_train.shape[1])
beta

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [120]:
# Gradient Descent
def lasso_regression(X_train, y_train, beta, LAMBDA, LEARNING_RATE, EPOCHS):
    for epoch in range(EPOCHS):
        y_pred = np.dot(X_train, beta)  # Predictions
        res = y_train - y_pred          # Residuals
        
        # Gradient computation
        gradient = ((-2 / X_train.shape[0]) * np.dot(X_train.T, res)) + (LAMBDA / X_train.shape[0]) * np.sign(beta)
        
        # Update weights
        beta -= LEARNING_RATE * gradient
        
        # Compute and print cost every 100 epochs
        if epoch % 100 == 0:
            cost = np.mean(np.square(res)) + LAMBDA * np.linalg.norm(beta, 1)
            print(f"Epoch: {epoch}, Cost: {cost}")
    return beta, cost

time_start = time()
beta, cost = lasso_regression(X_train, y_train, beta, LAMBDA, LEARNING_RATE, EPOCHS)
time_end = time()

print(f"Final Weights: {beta}")
print(f"MSE: {np.mean(np.square(y_train - np.dot(X_train, beta)))}")
print(f"Time taken: {time_end - time_start}")

Epoch: 0, Cost: 1.0000952498742655
Epoch: 100, Cost: 0.8852820688649577
Epoch: 200, Cost: 0.7902609792704005
Epoch: 300, Cost: 0.7116406718546322
Epoch: 400, Cost: 0.6466069493478077
Epoch: 500, Cost: 0.5928247744193663
Epoch: 600, Cost: 0.5483570715453853
Epoch: 700, Cost: 0.5115972530935621
Epoch: 800, Cost: 0.48121315311383983
Epoch: 900, Cost: 0.456100443046335
Final Weights: [-2.32022966e-08 -1.00175289e-01 -1.03725386e-01 -1.00615037e-01
 -1.11672068e-01  4.85094512e-02  8.61364409e-02  7.47838832e-02]
MSE: 0.3727437468362091
Time taken: 0.007403850555419922


In [121]:
# Stochastic Gradient Descent
def stochastic_lasso_regression(X_train, y_train, beta, LAMBDA, LEARNING_RATE, EPOCHS):
    for epoch in range(EPOCHS):
        np.random.seed(42)
        indices = np.random.permutation(X_train.shape[0])
        X_train = X_train[indices]
        y_train = y_train[indices]

        for i in range(X_train.shape[0]):
            # Select a random data point (stochastic)
            xi = X_train[i:i+1]
            yi = y_train[i:i+1]

            y_pred = np.dot(xi, beta)

            gradient = (-2 / xi.shape[0]) * np.dot(xi.T, (yi - y_pred)) + (LAMBDA / xi.shape[0]) * np.sign(beta)

            beta -= LEARNING_RATE * gradient

        if epoch % 100 == 0:
            y_pred = np.dot(X_train, beta)
            res = y_train - y_pred
            cost = np.mean(np.square(res)) + LAMBDA * np.linalg.norm(beta, 1)
            print(f"Epoch: {epoch}, Cost: {cost}")

    return beta, cost
time_start = time()
beta, cost = stochastic_lasso_regression(X_train, y_train, beta, LAMBDA, LEARNING_RATE, EPOCHS)
time_end = time()
print(f"Final Weights: {beta}")
print(f"MSE: {np.mean(np.square(y_train - np.dot(X_train, beta)))}")
print(f"R-Squared: {1 - np.sum(np.square(y_train - np.dot(X_train, beta))) / np.sum(np.square(y_train - np.mean(y_train)))}")
print(f"Time taken: {time_end - time_start}")

Epoch: 0, Cost: 0.38965240496977266
Epoch: 100, Cost: 0.2967260599039602
Epoch: 200, Cost: 0.29161811944441474
Epoch: 300, Cost: 0.29080737500056886
Epoch: 400, Cost: 0.2906873419685593
Epoch: 500, Cost: 0.2906558857750977
Epoch: 600, Cost: 0.29057975981634754
Epoch: 700, Cost: 0.29063380278603934
Epoch: 800, Cost: 0.2905825495441273
Epoch: 900, Cost: 0.2906094591164086
Final Weights: [ 1.48938300e-05 -9.47859836e-04 -4.91414166e-04 -3.83623248e-02
 -5.97733640e-01  5.35796403e-04  3.12598907e-01  9.02642658e-02]
MSE: 0.186492514248741
R-Squared: 0.813507485751259
Time taken: 2.1553826332092285


## **C. PUTTING EVERYTHING TOGETHER**

In [134]:
import numpy as np
from typing import Annotated

class LassoRegression:
    def __init__(self,
                 lambda_: Annotated[float, "Regularization strength"] = 0.1,
                 epochs: Annotated[int, "Number of training epochs"] = 1000,
                 learning_rate: Annotated[float, "Learning rate for optimization"] = 0.001,
                 ):
        self.lambda_ = lambda_
        self.epochs = epochs
        self.learning_rate = learning_rate

    def fit(self, X: np.ndarray, y: np.ndarray):
        _, n_features = X.shape
        self.beta = np.zeros(n_features)
        self.bias = 0
        for epoch in range(self.epochs):
            self.update_weights(X, y)

            if epoch % 100 == 0:
                self.cost += self.lambda_ * np.linalg.norm(self.beta, 1)
                print(f"Epoch: {epoch}, Cost: {self.cost}")

    def predict(self, X: np.ndarray):
        return np.dot(X, self.beta)
    
    def update_weights(self, X: np.ndarray, y: np.ndarray):
        n_samples, _ = X.shape
        y_pred = np.dot(X, self.beta)
        res = y - y_pred

        gradient = ((-2 / n_samples) * np.dot(X.T, res)) + (self.lambda_ / n_samples) * np.sign(self.beta)

        self.beta -= self.learning_rate * gradient
        self.bias -= self.learning_rate * np.mean(res)
        self.cost = np.mean(np.square(res)) + self.lambda_ * np.linalg.norm(self.beta, 1)

        return self.beta, self.bias, self.cost

In [135]:
model = LassoRegression(lambda_=0.1, epochs=7000, learning_rate=0.0001)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"MSE: {np.mean(np.square(y_test - y_pred))}")
print(f"R-Squared: {1 - np.sum(np.square(y_test - y_pred)) / np.sum(np.square(y_test - np.mean(y_test)))}")


Epoch: 0, Cost: 1.000190499748531
Epoch: 100, Cost: 0.8944748808130936
Epoch: 200, Cost: 0.8077585634135097
Epoch: 300, Cost: 0.7367199744720809
Epoch: 400, Cost: 0.6786085079094222
Epoch: 500, Cost: 0.6311471142772439
Epoch: 600, Cost: 0.5924515965367418
Epoch: 700, Cost: 0.5609635865152358
Epoch: 800, Cost: 0.5353948893672152
Epoch: 900, Cost: 0.514681273751248
Epoch: 1000, Cost: 0.49794410997113375
Epoch: 1100, Cost: 0.4844585281208707
Epoch: 1200, Cost: 0.47362699254943597
Epoch: 1300, Cost: 0.46495737539795173
Epoch: 1400, Cost: 0.45804476693950497
Epoch: 1500, Cost: 0.4525563892760035
Epoch: 1600, Cost: 0.44821908702781416
Epoch: 1700, Cost: 0.4448089576577394
Epoch: 1800, Cost: 0.4421427580500389
Epoch: 1900, Cost: 0.44007078545251854
Epoch: 2000, Cost: 0.43847098199260237
Epoch: 2100, Cost: 0.4372440544488312
Epoch: 2200, Cost: 0.4363094362539879
Epoch: 2300, Cost: 0.4356019480360047
Epoch: 2400, Cost: 0.43506903737467734
Epoch: 2500, Cost: 0.43466849870293867
Epoch: 2600, Cost